## OCT

Based on the structure of CART, the depth of OCT is also 1.

After respectively testing on alpha = 0.3, 0.5, 0.6 and epsilon = 0.01, 0.001, 0.0001, 10e-7, 10e-8, I find that the best accuracy is based on alpha = 0.6 and epsilon = 0.01.

In [42]:
alpha = 0.6
K = 2
Y_1 = np.zeros([N_1, K], dtype = int) - 1
Y_1[X_train_1.index, y_train_1.astype(int)] = 1
Y_1.shape

(518, 2)

In [43]:
import os
os.add_dll_directory(os.path.join(os.getenv('GUROBI_HOME'), 'bin'))

from gurobipy import *
model = Model('mip1')

<AddedDllDirectory('F:\\gurobi\\win64\\bin')>

Using license file C:\Users\Ke Xu\gurobi.lic
Academic license - for non-commercial use only - expires 2021-02-09


In [44]:
# declare decision variables

# d: [# branch nodes], whether a branch node applies a split or not
d = model.addVars(branch_nodes_1 ,vtype = GRB.BINARY, name = "d") # ∈ {0, 1}
# split criterion: ax < b / ax >== b
# a: [# branch nodes, p], b: [# branch nodes]
a = model.addVars(branch_nodes_1, P_1, vtype = GRB.BINARY, name = 'a') # ∈ {0, 1}
b = model.addVars(branch_nodes_1 ,vtype = GRB.CONTINUOUS, name = "b")

# l: [# leaf nodes], whether a leaf node contains any points or not
l = model.addVars(leaf_nodes_1, vtype = GRB.BINARY, name = "l") # ∈ {0, 1}
# z: [# points, # branch nodes], a point is assigned to which leaf node
z = model.addVars(N_1, leaf_nodes_1, vtype = GRB.BINARY, name = "z") # ∈ {0, 1}

# N_kt: [# labels, # leaf nodes], number of points labelled for class k in a leaf node
N_kt = model.addVars(K, leaf_nodes_1, vtype = GRB.INTEGER, name = "N_kt")
# N_t: [# leaf nodes], number of points in the leaf
N_t = model.addVars(leaf_nodes_1, vtype = GRB.INTEGER, name = "N_t")

# c_kt: [# labels, # leaf nodes], whether predicted label is label k for a leaf node
c_kt = model.addVars(K, leaf_nodes_1, vtype = GRB.BINARY, name = "c") # ∈ {0, 1}
# L: [# leaf nodes], misclassification loss for a leaf node
L = model.addVars(leaf_nodes_1, vtype = GRB.INTEGER, name = "L")

In [45]:
# warm start using the results of CART algorithm
for t in range(branch_nodes_1):
    a[t, initial_a_1[t]].start = 1
    b[t].start = initial_b_1[t]

model.update()

In [46]:
# baseline accuracy by predicting the dominant label for the whole dataset
L_hat = y_train_1.value_counts().max()/y_train_1.shape[0]
L_hat

0.5714285714285714

In [47]:
# declare the objective
model.setObjective(L.sum()/L_hat + alpha * d.sum(), GRB.MINIMIZE)

In [48]:
def get_parent(i, depth = 1):
    assert i > 0, "No parent for Root"
    assert i <= 2 ** (depth + 1) - 1, "Error! Total: {0}; i: {1}".format(
        2 ** (depth + 1) - 1, i)
    return int((i - 1)/2)

In [49]:
# constraint set 1 
for t in range(branch_nodes_1):
    model.addConstr(a.sum(t, '*') == d[t]) # sum(a_tj for j in P) = d_t
    b[t].setAttr(GRB.Attr.LB, 0) # b_t >= 0
    model.addConstr(b[t] <= d[t]) # b_t <= d_t
    model.addConstr(d[t] == 1) # d_t = 1 (assume all the branch applies a split)

# constraint set 2
for t in range(1, branch_nodes_1): # exception: root
    model.addConstr(d[t] <= d[get_parent(t)]) # d_t <= d_p(t)

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

In [50]:
# constraint set 3
for i in range(N_1):
    model.addConstr(z.sum(i, '*') == 1) # sum(z_it for t in T_L)

# constraint set 4
N_min = 1
for t in range(leaf_nodes_1):
    model.addConstr(l[t] == 1) # l_t == 1 (assume leaf contains points)
    for i in range(N_1):
        model.addConstr(z[i, t] <= l[t]) # z_it <= l_t
    model.addConstr(z.sum('*', t) >= N_min * l[t]) # sum(z_it for i in N) >= N_min * l_t

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

...

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

In [51]:
depth = 1
all_branch_nodes = list(reversed(range(branch_nodes_1)))
depth_dict = {}
for i in range(depth):
    depth_dict[i] = sorted(all_branch_nodes[-2**i:])
    for j in range(2**i):
        all_branch_nodes.pop()
depth_dict

0

{0: [0]}

In [52]:
all_leaf_nodes = list(range(leaf_nodes_1))
branch_dict = {}
for i in range(branch_nodes_1):
    for k in range(depth):
        if i in depth_dict[k]:
            floor_len = len(depth_dict[k])
            step = 2**depth // floor_len
            sliced_leaf = [all_leaf_nodes[i:i+step] for i in range(0, 2**depth, step)]
            idx = depth_dict[k].index(i)
            branch_dict[i] = sliced_leaf[idx]
        else:
            continue
branch_dict

{0: [0, 1]}

In [53]:
epsilon = 0.01
# constraint set 5
for i in range(N_1):
    for tl in range(leaf_nodes_1):
        for tb in range(branch_nodes_1):
            if tl in branch_dict[tb]:
                length = len(branch_dict[tb])
                idx = branch_dict[tb].index(tl)
                # left-branch ancestors: 
                # np.dot(a_m.T, (x_i + mu))<= b_m + (1 + mu)(1- z_it)
                if idx+1 <= length//2:
                    model.addConstr(
                        sum(a.select(tb, '*') * X_train_1.iloc[i, :]) + epsilon 
                        <= b[tb] + (1 + epsilon) * (1 - z[i, tl]))
                # right-branch ancestors: 
                # np.dot(a_m.T, x_i) >= b_m - (1- z_it)
                elif idx+1 > length//2:
                    model.addConstr(
                        sum(a.select(tb, '*') * X_train_1.iloc[i, :]) 
                        >= b[tb] - (1 - z[i, tl]))
            else:
                continue

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

...

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

In [54]:
# constraint set 6, 7 & 8
for t in range(leaf_nodes_1):
    
    # constraint set 8
    model.addConstr(L[t] >= 0) # L_t >= 0
    for k in range(K):
        # L_t >= N_t - N_kt - n(1 - c_kt)
        model.addConstr(L[t] >= N_t[t] - N_kt[k, t] - N_1 * (1 - c_kt[k, t])) 
        # L_t <= N_t - N_kt + n * c_kt
        model.addConstr(L[t] <= N_t[t] - N_kt[k, t] + N_1 * c_kt[k, t]) 
        
        # constraint set 6
        # N_kt = 1/2 sum((1 + Y_ik)z_it for i in N)
        model.addConstr(N_kt[k, t] == 1/2 * sum(z.select('*', t) * (Y_1[:, k] + 1))) 
    
    model.addConstr(N_t[t] == z.sum('*', t)) # N_t = sum(z_it for i in n)
    # constraint set 7
    model.addConstr(c_kt.sum('*', t) == l[t]) # l_t = sum(c_kt for k in K)

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

...

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

<gurobi.Constr *Awaiting Model Update*>

In [56]:
model.Params.timelimit = 60*10

Changed value of parameter timelimit to 600.0
   Prev: 300.0  Min: 0.0  Max: inf  Default: inf


In [57]:
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2615 rows, 1089 columns and 17981 nonzeros
Model fingerprint: 0xee96ac73
Variable types: 1 continuous, 1088 integer (1080 binary)
Coefficient statistics:
  Matrix range     [1e-05, 5e+02]
  Objective range  [6e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]

User MIP start produced solution with objective 130.1 (0.03s)
Loaded user MIP start with objective 130.1

Presolve removed 1567 rows and 528 columns
Presolve time: 0.05s
Presolved: 1048 rows, 561 columns, 12014 nonzeros
Variable types: 1 continuous, 560 integer (555 binary)

Root relaxation: objective 6.000000e-01, 7 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.60000    0  155  130.10000    0.60000   10

In [58]:
print('Obj:', model.objVal)

Obj: 130.1


In [59]:
coef_a = np.zeros([branch_nodes_1, P_1], dtype = int)
coef_b = np.zeros(branch_nodes_1)

In [60]:
for i in range(branch_nodes_1):
    b = model.getVarByName('b' + '[' + str(i) + ']')
    coef_b[i] = b.x
    for j in range(P_1):
        a = model.getVarByName('a' + '[' + str(i) + ',' + str (j) + ']')
        coef_a[i, j] = int(a.x)

In [61]:
coef_a

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]])

In [63]:
coef_b

array([0.5])

In [64]:
_ , a_idx = np.where(coef_a == 1)
a_idx = a_idx.tolist()

OCT_a = []
for i in range(len(feature_1)):
    if i in np.where(feature_1 == -2)[0]:
        OCT_a.append(-2)
    else:
        OCT_a.append(a_idx[0])
        a_idx.pop(0)

OCT_b = []
tmp_b = coef_b.tolist()
for i in range(len(threshold_1)):
    if i in np.where(threshold_1 == -2)[0]:
        OCT_b.append(-2)
    else:
        OCT_b.append(round(tmp_b[0], 2))
        tmp_b.pop(0)

32

0.5

In [65]:
OCT_a
OCT_b

[32, -2, -2]

[0.5, -2, -2]

In [66]:
OCT_rules = {}
for leaf in paths_1:
    OCT_rules[leaf] = get_rule(
        paths_1[leaf], children_left_1, tuple(df_1_final.columns[0:-1]), OCT_a, OCT_b)

In [67]:
OCT_rules

{1: "('A9_f' <= 0.50)", 2: "('A9_f' > 0.50)"}

In [68]:
coef_c = np.zeros([K, leaf_nodes_1], dtype = int)

for i in range(K):
    for j in range(leaf_nodes_1):
        c = model.getVarByName('c' + '[' + str(i) + ',' + str (j) + ']')
        coef_c[i,j] = int(c.x)

In [69]:
coef_c

array([[0, 1],
       [1, 0]])

In [70]:
k_idx, t_idx = np.where(coef_c == 1)
labels = np.zeros(leaf_nodes_1, dtype = int) - 1
for i in range(len(k_idx)):
    labels[t_idx[i]] = k_idx[i]

In [71]:
labels

array([1, 0])

In [72]:
y_hat = np.hstack([
    np.reshape(y_train_1.values, (N_1, 1)), 
    np.zeros([N_1, 1], dtype = int)])

In [73]:
num_nodes = 0
for i in range(branch_nodes_1): 
    d = model.getVarByName('d' + '[' + str(i) + ']')
    num_nodes += int(d.x)
num_nodes

1

In [74]:
# initialize 
init = np.array([], dtype = int).reshape(0, P_1)
nodes = {}
for i in range(num_nodes * 2):
    nodes[i] = init
# split
# split
for i in range(N_1):
    if np.dot(coef_a[0,:], np.transpose(X_train_1.iloc[i,:])) <= coef_b[0]:
        nodes[0] = np.vstack([X_train_1.iloc[i,:], nodes[0]])
        
#         if np.dot(coef_a[1,:], np.transpose(X_train_1.iloc[i,:])) <= coef_b[1]:
#             nodes[2] = np.vstack([X_train_1.iloc[i,:], nodes[2]])
        y_hat[i,1] = labels[0]
                
#         elif np.dot(coef_a[1,:], np.transpose(X_train_1.iloc[i,:])) > coef_b[1]:
#             nodes[3] = np.vstack([X_train_1.iloc[i,:], nodes[3]])
#             y_hat[i,1] = labels[1]
            
    elif np.dot(coef_a[0,:], np.transpose(X_train_1.iloc[i,:])) > coef_b[0]:
        nodes[1] = np.vstack([X_train_1.iloc[i,:], nodes[1]])
        
#         if np.dot(coef_a[2,:], np.transpose(X_train_1.iloc[i,:])) <= coef_b[2]:
#             nodes[4] = np.vstack([X_train_1.iloc[i,:], nodes[4]])
        y_hat[i,1] = labels[1]
           
#         elif np.dot(coef_a[2,:], np.transpose(X_train_1.iloc[i,:])) > coef_b[2]:
#             nodes[5] = np.vstack([X_train_1.iloc[i,:], nodes[5]])
#             y_hat[i,1] = labels[3]

In [76]:
performance_in = 1 - sum(np.abs(y_hat[:,1] - y_hat[:,0])) / N_1

In [77]:
for i in range(len(labels)):
    print('\nNode {}'.format(str(i+7)))
    print('Predicted label: {}'.format(str(labels[i])))
    print('No. of obs.: {}'.format(nodes[i].shape[0]))


Node 7
Predicted label: 1
No. of obs.: 264

Node 8
Predicted label: 0
No. of obs.: 254


In [78]:
N_prime, P = X_test_1.shape

In [79]:
y_predict = np.hstack([
    np.reshape(y_test_1.values, (N_prime, 1)), 
    np.zeros([N_prime, 1], dtype = int)])

In [80]:
# initialize 
init = np.array([], dtype = int).reshape(0, P)
nodes = {}
for i in range(num_nodes * 2):
    nodes[i] = init

# split
for i in range(N_prime):
    if np.dot(coef_a[0,:], np.transpose(X_test_1.iloc[i,:])) <= coef_b[0]:
        nodes[0] = np.vstack([X_test_1.iloc[i,:], nodes[0]])
        
#         if np.dot(coef_a[1,:], np.transpose(X_test_1.iloc[i,:])) <= coef_b[1]:
#             nodes[2] = np.vstack([X_test_1.iloc[i,:], nodes[2]])
#             y_predict[i,1] = labels[0]
                
#         elif np.dot(coef_a[1,:], np.transpose(X_test_1.iloc[i,:])) > coef_b[1]:
#             nodes[3] = np.vstack([X_test_1.iloc[i,:], nodes[3]])
        y_predict[i,1] = labels[0]
            
    elif np.dot(coef_a[0,:], np.transpose(X_test_1.iloc[i,:])) > coef_b[0]:
        nodes[1] = np.vstack([X_test_1.iloc[i,:], nodes[1]])
        
#         if np.dot(coef_a[2,:], np.transpose(X_test_1.iloc[i,:])) <= coef_b[2]:
#             nodes[4] = np.vstack([X_test_1.iloc[i,:], nodes[4]])
#             y_predict[i,1] = labels[2]
           
#         elif np.dot(coef_a[2,:], np.transpose(X_test_1.iloc[i,:])) > coef_b[2]:
#             nodes[5] = np.vstack([X_test_1.iloc[i,:], nodes[5]])
        y_predict[i,1] = labels[1]

In [82]:
performance_out = 1 - sum(np.abs(y_predict[:,1] - y_predict[:,0])) / N_prime

In [83]:
for i in range(len(labels)):
    print('\nNode {}'.format(str(i+7)))
    print('Predicted label: {}'.format(str(labels[i])))
    print('No. of obs.: {}'.format(nodes[i+0].shape[0]))


Node 7
Predicted label: 1
No. of obs.: 97

Node 8
Predicted label: 0
No. of obs.: 75


###  Rules and performance of OCT

In [88]:
print('The decision rules in OCT is ', OCT_rules)
print('\nThe in-sample performance in OCT is', performance_in)
print('\nThe out-of-sample performance in OCT is',  performance_out)

The decision rules in OCT is  {1: "('A9_f' <= 0.50)", 2: "('A9_f' > 0.50)"}

The in-sample performance in OCT is 0.8571428571428572

The out-of-sample performance in OCT is 0.8488372093023255
